In [588]:
from function import *
import numpy as np
import networkx as nx
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [589]:
# Using Random Data

# Number of data
N = 10
# Number of features
d = 3
# Control precision bit of the output 
precision = 10
# Size of the QUBO matrix
dim = d * (2 * precision)

# Generate Random Data
X = np.random.rand(N, d)
Y = np.random.rand(N)

# If you want to perform polynomial regression
# X, d, dim = polynomialForm(X, d, dim, precision)

print(X)
print(Y)
print(d, dim)

[[1.00000000e+00 4.78261215e-01 9.66036379e-02 3.00611683e-01
  2.28733790e-01 4.62017732e-02 1.43770909e-01 9.33226286e-03
  2.90401822e-02 9.03673840e-02]
 [1.00000000e+00 2.96969956e-01 2.89443528e-01 4.41884829e-02
  8.81911545e-02 8.59560317e-02 1.31226518e-02 8.37775560e-02
  1.27900704e-02 1.95262202e-03]
 [1.00000000e+00 6.57691649e-01 2.68446643e-02 3.49217469e-01
  4.32558305e-01 1.76555115e-02 2.29677413e-01 7.20635999e-04
  9.37462571e-03 1.21952841e-01]
 [1.00000000e+00 2.91861957e-01 5.70982551e-01 1.26912996e-01
  8.51834018e-02 1.66648085e-01 3.70410753e-02 3.26021073e-01
  7.24651061e-02 1.61069085e-02]
 [1.00000000e+00 6.83789175e-01 8.59538950e-01 3.00516997e-01
  4.67567636e-01 5.87743430e-01 2.05490270e-01 7.38807207e-01
  2.58306064e-01 9.03104656e-02]
 [1.00000000e+00 7.78361504e-01 1.67817080e-01 6.26501861e-01
  6.05846631e-01 1.30622354e-01 4.87644931e-01 2.81625722e-02
  1.05137713e-01 3.92504582e-01]
 [1.00000000e+00 1.28216113e-01 8.11853780e-01 7.10640027e

In [590]:
XtX = np.matmul(X.T, X) # Covariance matrix
XtY = np.matmul(X.T, Y) # Relation between Features and Y

# print(XtX)
# print(XtY)

In [591]:
# Creating the graph
G = nx.Graph()
G.add_edges_from([(i, j) for i in range(dim) for j in range(i + 1, dim)])

# The matrix where we add the objective and the constraint (QUBO matrix)s
Q = defaultdict(int)

print(G)
print(Q)

Graph with 200 nodes and 19900 edges
defaultdict(<class 'int'>, {})


In [592]:
generateQuboMatrix(XtX, XtY, precision, Q, d)

In [593]:
sampleset = sampling(Q)
distributions, weights, error = solve(sampleset, dim, precision, X,Y, d)

# print(distributions)
print(weights)
print("MSE Quantum : ", error)

[ 0.62890625 -0.5         0.28710938  1.25       -0.05859375  0.875
 -1.16210938 -1.         -0.24609375 -0.24414062]
MSE Quantum :  0.03276511021242938


In [594]:
clf = LinearRegression()
clf.fit(X, Y)
print(clf.coef_)
print("MSE Sklearn: ", mse(clf.predict(X),Y))

[  0.         -12.80509922   1.29995121  20.12329745  13.45344005
  10.89241529 -22.75069703  -2.55562076 -20.59188476   0.24048282]
MSE Sklearn:  1.2894178014870318e-29


In [595]:
diff = 100*(error - mse(clf.predict(X),Y))/mse(clf.predict(X),Y)

print("Sklearn perform ", diff, "% better than Quantum")

Sklearn perform  2.5410778550321502e+29 % better than Quantum


In [596]:
# poly = PolynomialFeatures(degree=2)
# X_poly = poly.fit_transform(X)

# clf = LinearRegression()
# clf.fit(X_poly, Y)
# print(clf.coef_)
# print("MSE Sklearn: ", mse(clf.predict(X),Y))